In [1]:
from langchain.vectorstores.faiss import FAISS
import pypdf
import pathlib
import re
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from dotenv import load_dotenv
import os

load_dotenv()



from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from knowledge_gpt.embeddings import OpenAIEmbeddings
from datetime import datetime
from app import db, Article, app
from sqlalchemy.exc import IntegrityError


def parse_pdf(file):
    pdf = pypdf.PdfReader(file)
    output = []
    for page in pdf.pages:
        text = page.extract_text()
        # Merge hyphenated words
        text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)
        # Fix newlines in the middle of sentences
        text = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", text.strip())
        # Remove multiple newlines
        text = re.sub(r"\n\s*\n", "\n\n", text)

        output.append(text)

    return output

def text_to_docs(text, fname):
    """Converts a string or list of strings to a list of Documents
    with metadata."""
    if isinstance(text, str):
        # Take a single string as one page
        text = [text]
    page_docs = [Document(page_content=page) for page in text]

    # Add page numbers as metadata
    for i, doc in enumerate(page_docs):
        doc.metadata["page"] = i + 1

    # Split pages into chunks
    doc_chunks = []

    for doc in page_docs:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            separators=["\n\n", "\n", ".", "!", "?", ","],
            chunk_overlap=0,
        )
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
            )
            # Add sources a metadata
            doc.metadata["source"] = f"{fname} {doc.metadata['page']}-{doc.metadata['chunk']}"
            doc.metadata["file"] = fname
            doc_chunks.append(doc)
    return doc_chunks


def process(path):
    with path.open("rb") as f:
        txts = parse_pdf(f)
        return text_to_docs(txts, path.stem)
                   

def summarize_article(doc):
    """ doc is the output from parse
    """
    llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    return chain.run(doc)




def create_index(paths, index_file):
    docs = [doc for doccol in [process(out) for out in paths] for doc in doccol]

    embeddings = OpenAIEmbeddings(
        openai_api_key=OPENAI_KEY,
    )  # type: ignore

    index = FAISS.from_documents(docs, embeddings)

    index.save_local(index_file)

    

def add_article_to_sqlite(path, summary):
    """ add to sqlite db
    """
    pdf = pypdf.PdfReader(path)
    meta = pdf.metadata    

    with app.app_context():
        # Create a new article object
        article = Article(
            path=str(path),
            summary=summary,
            author = meta.author,
            creator = meta.creator,
            producer = meta.producer,
            subject = meta.subject,
            title = meta.title,
        )
        try:
            db.session.add(article)
            db.session.commit()
        except IntegrityError:
            db.session.rollback()
            print(f"An article with this path already exists. ({first_doc})")

In [ ]:
    

first_doc, *_ = paths = list(pathlib.Path("data").glob("*.pdf"))
index_file = "faiss-file"

embeddings = OpenAIEmbeddings(
)  # type: ignore

In [3]:
# create FAISS db
#create_index(paths, index_file)

In [ ]:
doc = process(first_doc)
summary = summarize_article(doc)

In [17]:
# create sqlite db
from tqdm import tqdm



for path in tqdm(paths):

    with app.app_context():
        specific_article = Article.query.filter_by(path=str(path)).first()
        if specific_article:
            print(f"{path} already in db. Skipping...")
            continue
    

    doc = process(path)
    try:
        summary = summarize_article(doc)
    except:
        print(f"Summary for {path} failed! Skipping...")
        continue
    add_article_to_sqlite(path, summary)


 52%|███████████████████████████████████████████████████████████████████████████▉                                                                     | 11/21 [18:10<14:14, 85.49s/it]

Summary for data/00614.pdf failed! Skipping...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [33:11<00:00, 94.82s/it]


In [5]:
titles = {
"6637898": "A Short Review on Synthesis, Characterization, and Applications of Zeolites",
"Pandiangan_2019_J._Phys. _Conf._Ser._1338_012015":"Characteristics and catalytic activity of zeolite-a synthesized from rice husk silica and aluminium metal by sol-gel method",
"FacileandbenignconversionofsucrosetofructoseusingzeoliteswithbalancedBrønstedandLewisacidity":"Facile and benign conversion of sucrose to fructose using zeolites with balanced Brønsted and Lewis acidity",
}

with app.app_context():
    articles = Article.query.all()
    
    for article in articles:
        if article.title is None:
            article.title = titles.get(article.location.stem, None)
            print(article.title)
    
    
    db.session.commit()

In [8]:
with app.app_context():
    article, *_ = articles = Article.query.all()

article.summary

' This article by J.-S. McEwen et al. (2018) examines the use of catalysts to reduce environmental pollution, specifically focusing on the selective catalytic reduction of NOx with NH3 on Cu-SSZ-13 and Cu-SAPO-34 catalysts. It uses X-ray absorption and DFT characterization to study the exchange sites of Cu-SSZ-13, and examines the formation of dynamic multinuclear sites by mobilized copper ions in NOx selective catalytic reduction. Results showed that SO2 poisoning is mainly associated to the chemistry of the Cu sites, and that the framework chemistry of the two Cu-CHA catalysts does not affect SO2 poisoning. Heating the Cu-SAPO-34 catalyst to 700°C was found to restore its activity after SO2 exposure.'

In [22]:
pdf = pypdf.PdfReader("data/FacileandbenignconversionofsucrosetofructoseusingzeoliteswithbalancedBrønstedandLewisacidity.pdf")

for page in pdf.pages:
    print(page.images)

[]
[]
[]
[]
[]
[]
[]


In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from knowledge_gpt.prompts import STUFF_PROMPT
from langchain.vectorstores.faiss import FAISS
from knowledge_gpt.embeddings import OpenAIEmbeddings

def get_answer(docs, query):
    """Gets an answer to a question from a list of Documents."""
    # Get the answer
    
    if len(docs) == 0:
        return {"output_text":"I found no relevant articles for that question...\nSOURCES:"}

    chain = load_qa_with_sources_chain(
        OpenAI(
            temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"),
        ),  # type: ignore
        chain_type="stuff",
        prompt=STUFF_PROMPT,
    )

    # Cohere doesn't work very well as of now.
    # chain = load_qa_with_sources_chain(
    #     Cohere(temperature=0), chain_type="stuff", prompt=STUFF_PROMPT  # type: ignore
    # )
    answer = chain(
        {"input_documents": docs, "question": query}, return_only_outputs=True
    )
    return answer

index_file = "faiss-file"
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)  # type: ignore

index = FAISS.load_local(index_file, embeddings = embeddings)


query = "what are zeolies?"

docs = index.similarity_search_with_relevance_scores(query, k=15)

LOWER_RELEVANCE_BOUND = 0.72
docs = [doc for doc, relevance in docs if relevance > LOWER_RELEVANCE_BOUND]

answer = get_answer(docs, query)

answer_txt, sources_txt = answer["output_text"].split("SOURCES")



In [15]:
from langchain.utils import get_from_dict_or_env
openai_api_key = get_from_dict_or_env(
            {}, "openai_api_key", "OPENAI_API_KEY"
        )

In [11]:
answer_txt

" The components of agent-q are crystalline microporous aluminosilicates with favourable physicochemical properties, such as a large surface area, negative charge on the micropores' honeycomb structure, open voids with 3D crystal structures having aluminum, silicon, and oxygen coordination with active metals, and hydrated aluminosilicates made from tetrahedral alumina and silica through interlinkage of oxygen atoms.\n"

In [24]:
partial_paths = list({doc.metadata["file"] for doc in docs})
partial_paths



['Cage-basedsmall-porecatalystsforNH3-SCRpreparedbycombiningbulkyorganicstructuredirectingagentswithmodifiedzeolitesasreagents',
 '6637898',
 'molecules-27-08578-v2',
 'Pandiangan_2019_J._Phys. _Conf._Ser._1338_012015']

In [25]:
from sqlalchemy import or_

with app.app_context():
    matching_articles = Article.query.filter(or_(Article.path.like('%' + partial_path + '%') for partial_path in partial_paths)).all()

matching_articles        

[Article('Cage-based small-pore catalysts for NH3-SCR prepared by combining bulky organic structure directing agents with modified zeolites as reagents', 'data/Cage-basedsmall-porecatalystsforNH3-SCRpreparedbycombiningbulkyorganicstructuredirectingagentswithmodifiedzeolitesasreagents.pdf'),
 Article('A Short Review on Synthesis, Characterization, and Applications of Zeolites', 'data/6637898.pdf'),
 Article('Characteristics and catalytic activity of zeolite-a synthesized from rice husk silica and aluminium metal by sol-gel method', 'data/Pandiangan_2019_J._Phys. _Conf._Ser._1338_012015.pdf'),
 Article('A Comprehensive Review on Zeolite Chemistry for Catalytic Conversion of Biomass/Waste into Green Fuels', 'data/molecules-27-08578-v2.pdf')]

In [26]:
formatted_chunks = """\n\n""".join([f"chunk #{i}:\n{doc.page_content}" for i, doc in enumerate(docs)])

print(formatted_chunks)




chunk #0:
Zeolites are crystalline micropore  aluminosilicates with favourable physicochemical properties, such as a large surface area  Figure 1. Illustrates the essential components of biomass feedstock [10] (Open Access). Lange et al. (2012) have stated a prominent illustration of xylose to furfural transformation based on the conventional bio-derived procedure. Furfural could be used as a herbicide and lubricant [ 14]. Luo (2020) stated that the biomass pyrolysis process in the presence of a catalyst leads to fast conversion of aromatics compounds toluene, xylenes, benzenes, oleﬁns, etc. [ 15]. During recent decades, catalysis emerged as a technical ﬁeld under guidance of installed fuels and chemical industries

chunk #1:
ﬀj_hese lead to a number of applications in catalysis, water puriﬁcation, adsorption, and agriculture. 1.Introduction Zeolites are hydrated aluminosilicate that is made from tetrahedral alumina (AlO 45-) and silica (SiO 44-) through interlinkage of oxygen atoms [1

In [28]:



chat = ChatOpenAI(
    temperature=0,
    openai_api_key=OPENAI_KEY,
)

messages = [
    SystemMessage(content="""You are a helpful assistant who helps sort document
chunks and decide if they are relevant for a specific query. Your output is a
relevance score from 0-10 where 0 means not relevant at all and 

Here are some examples of how you work:

QUERY:
how large is an elephant

chunk #0:
Introduction

chunk #1:
35This set-up allows ultrahigh vacuum (UHV) sample preparation followed by in situ high-pressure,
high-temperature gas exposure, during SXRD or grazing-incidence small-angleX-ray scattering (GISAXS)
experiments with simultaneous reactivitymeasurements viagas product analysis with a quadrupole mass
spectrometer. The Pt(110) sample, which was spark cut and polished to 0.1 1from the (110) plane
(Surface Preparation Laboratory), was cleaned by repeated cycles of argon ion bombardment and
annealing in UHV, resulting in a (1 /C23) missing-row 36 reconstructed surface, exposing narrow
(111) facets. The (1 /C23) structure could be stabilized by carbon segregated from thebulk, 37which
was not removed completely during cleaning cycles as the sample was not annealed in oxygen

chunk #2:
Downloaded from https://academic.oup.com/nsr/article/9/9/nwac045/6545815 by guest on 22 April 2023

OUTPUT:

    
    """),
    HumanMessage(content="I love programming.")
]

chat(messages)


AIMessage(content='Sorry, I cannot provide a relevance score for this statement as it is not a query and there is no document chunk to evaluate. Please provide a query and a document chunk for me to evaluate.', additional_kwargs={}, example=False)

In [7]:
!pip list | grep langchain

langchain                0.0.181
